In [1]:
import pandas as pd
import numpy as np

Sbase=100

def createMatrixX(lines,NB):
    X=np.zeros((NB,NB))
    for i in range(len(lines)):
        nodo_i=int(lines[i,0])
        nodo_j=int(lines[i,1])
        X[nodo_i,nodo_j]=lines[i,3]
        X[nodo_j,nodo_i]=lines[i,3]
    return X

def createMatrixB(X):
    B=X.copy()
    for i in range(len(B)):
        for j in range(len(B)):
            if i==j:
                B[i,i]=sum(1/k for k in X[i,:] if k!=0)
            else:
                if X[i,j]!=0:
                    B[i,j]=-1/X[i,j]
                else:
                    B[i,j]=0
    return B

def getPd(loads,NB):
    Pd=[0]*NB
    for i in range(NB):
        ii=int(loads[i,0])
        Pd[ii]=loads[i,1]/Sbase
    return Pd

def cost_der(x):
    g=np.zeros_like(x)
    n=len(Beta)
    for i in range(n):
        g[i]=Beta[i]
    g[n:]=0
    return g

def cost_hess(x):
    h=np.zeros((len(x),len(x)))
    return h

def makeConstraint(NB,NL,NG,slack,file,X,B,Pd,Cap):
    
    dflines=pd.read_excel(file,'Lines')
    lines=dflines.values
    dfgenerators=pd.read_excel(file,'Generators')
    generators=dfgenerators.values
    binf=np.zeros(NB+NL)
    bsup=np.zeros(NB+NL)
    A=np.zeros((NB+NL,NG+NB))
    r=mapGen(generators,NG)
    A[0:NB,0:NG]=r
    for i in range(NB):
        for j in range(NG,NG+NB):
            A[i,j]=B[i,j-NG]
    #Constraints of flows (angles)
    bsup[0:NB]=np.copy(Pd)
    binf[0:NB]=np.copy(Pd)
    k=0 #number of contraints of flows
    for line in lines:
        i=int(line[0])
        j=int(line[1])
        A[NB+k,NG+i]=1
        A[NB+k,NG+j]=-1
        bsup[NB+k]=Cap*X[i,j]
        binf[NB+k]=-Cap*X[i,j]
        k=k+1
    #Take out slack angle (because theta0 = 0)
    aux=[i for i in range(NG+NB) if i!=(NG+slack)]
    return A[:,aux],binf,bsup
    

def makeBounds():
    lb=[0]*(NG+NB-1) #slack angle=0
    ub=[0]*(NG+NB-1)
    for i in range(NG):
        lb[i]=Pmin[i]
        ub[i]=Pmax[i]
    for i in range(NG,NG+NB-1):
        lb[i]=-1
        ub[i]=1
    bounds = Bounds(lb,ub)
    return bounds

def printFlows(res):
    print('Flows [pu]')
    Theta=res.x[NG:]
    Theta=np.insert(Theta,0,0,axis=0)
    for i in range(1,len(Theta)):
        Thetai=Theta[i-1]
        Thetaj=Theta[i]
        Fij=-(Thetai-Thetaj)/X[i-1,i]
        print('P'+str(i-1)+str(i)+' = ',round(Fij,2))

def printBalance(res):
    print('Balance [pu]')
    print('Generation = ',sum(res.x[0:NG]))
    print('Demand = ',sum(Pd))
    
def printGeneration(res):
    print('Generation by Units [pu]')
    G=res.x[0:NG]
    for i in range(NG):
        print('PG'+str(i)+' = ',round(G[i],2))
        
def printAngles(res):
    print('Angles [pu]')
    Theta=res.x[NG:]
    Theta=np.insert(Theta,0,0,axis=0)
    for i in range(len(Theta)):
        print('Theta'+str(i)+' = ',round(Theta[i],4))
        
def printCosts(res):
    print('System Costs [USD]')
    print('Cost = ',round(np.dot(res.x[0:NG],Beta),2))
    
def makeStartPoint():
    y0=np.zeros(NG+NB-1)
    for i in range(NG):
        y0[i]=0.1
    for i in range(NG,NG+NB-1):
        y0[i]=0
    return y0

def readsystem(file):
    #READ VALUES
    dflines=pd.read_excel(file,'Lines')
    lines=dflines.values
    dfloads=pd.read_excel(file,'Loads')
    loads=dfloads.values
    dfbus=pd.read_excel(file,'Busbar')
    bus=dfbus.values
    dfgenerators=pd.read_excel(file,'Generators')
    generators=dfgenerators.values

    #Number of lines
    NL=len(lines)
    #Number of busbar
    NB=bus[0,0]
    #Number og generators
    NG=generators.shape[0]

    #TRANSMISSION SYSTEM
    #Create reactance matrix
    X=createMatrixX(lines,NB)
    B=createMatrixB(X)

    #LOAD CONSUMERS
    #Power demand by nodes
    Pd=getPd(loads,NB)

    #GENERATORS PARAMETERS
    #Costs
    Alpha, Beta, Gamma=getCosts(generators,NG)
    #Pmax, Pmin
    Pmin,Pmax=getPmaxmin(generators,NG)
    return NB,NL,NG,Pd, Alpha, Beta, Gamma,X,B,Pmin,Pmax

def getCosts(generators,NG):
    Alpha = [0]*NG
    Beta = [0]*NG
    Gamma = [0]*NG
    for i in range(NG):
        Alpha[i]=generators[i,3]
        Beta[i]=generators[i,4]
        Gamma[i]=generators[i,5]
    return Alpha, Beta, Gamma
    
def getPmaxmin(generators,NG):
    Pmax=[0]*NG
    Pmin=[0]*NG
    for i in range(NG):
        Pmin[i]=generators[i,6]/Sbase
        Pmax[i]=generators[i,7]/Sbase
    return Pmin,Pmax

def mapGen(generators,NG):
    r=np.zeros((NB,NG))
    for i in range(NG):
        j=int(generators[i,0])
        r[j,i]=1
    return r

# Import Data

In [2]:
import pandas as pd
import numpy as np
from __future__ import print_function
from ortools.linear_solver import pywraplp

#Cargar datos
file='Data_radial_2.xlsx'
NB,NL,NG,Pd, Alpha, Beta, Gamma,X,B,Pmin,Pmax = readsystem(file)

# Solution

In [3]:
from scipy.optimize import Bounds,LinearConstraint, minimize

bounds = makeBounds()

slack=0
Cap=0.1 #Use Cap=0.0001 to reduce flows to 0

#from Ax=b, A matrix
A,binf,bsup=makeConstraint(NB,NL,NG,slack,file,X,B,Pd,Cap)
linear_constraint = LinearConstraint(A,binf,bsup)         


def fo(y):
    return np.dot(y[0:NG],Beta)

y0 = makeStartPoint()
res = minimize(fo, y0, method='trust-constr',jac=cost_der,
               hess=cost_hess, constraints=linear_constraint,
               options={'verbose': 1,'maxiter':1024,'xtol':1e-8}, bounds=bounds)


#PRINT RESULTS
print('--------------------------')
printBalance(res)
print('--------------------------')
printGeneration(res)
print('--------------------------')
printFlows(res)
print('--------------------------')
printAngles(res)
print('--------------------------')
printCosts(res)

`gtol` termination condition is satisfied.
Number of iterations: 21, function evaluations: 19, CG iterations: 24, optimality: 8.67e-10, constraint violation: 3.11e-15, execution time: 0.057 s.
--------------------------
Balance [pu]
Generation =  2.0000000000000004
Demand =  2.0
--------------------------
Generation by Units [pu]
PG0 =  0.5
PG1 =  0.6
PG2 =  0.9
PG3 =  0.0
--------------------------
Flows [pu]
P01 =  0.1
--------------------------
Angles [pu]
Theta0 =  0.0
Theta1 =  0.01
--------------------------
System Costs [USD]
Cost =  113.01
